In [3]:
# 1) Design model ( input, output size, forward pass)
# 2) Construct loss and optimizer
# 3) Training loop
#    - forward pass: compute prediction
#    - backward pass: gradients
#    - update weights

In [4]:
import torch

In [5]:
import torch.nn as nn

In [6]:
import numpy as np

In [7]:
from sklearn import datasets

In [8]:
import matplotlib.pyplot as plt

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [11]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
print(f"X shape = {X.shape}")
print(f"y shape = {y.shape}")

X shape = (569, 30)
y shape = (569,)


In [12]:
n_samples, n_features = X.shape

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=42)

In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [16]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [17]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

In [18]:
model = LogisticRegression(n_features)

In [19]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

In [20]:
num_epochs = 100

In [21]:
for epoch in range(num_epochs):
    # forward
    y_predicted = model(X_train)
    
    # loss
    loss = criterion(y_predicted, y_train)
    
    # backward
    loss.backward()
    
    # updates
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        [w, b] = model.parameters()
        print(f"epoch {epoch + 1}, w = {w[0][0].item():.3f}, loss = {loss.item():.8f}")
    

epoch 10, w = -0.081, loss = 0.66980559
epoch 20, w = -0.109, loss = 0.55002642
epoch 30, w = -0.131, loss = 0.47467309
epoch 40, w = -0.149, loss = 0.42293566
epoch 50, w = -0.165, loss = 0.38500568
epoch 60, w = -0.179, loss = 0.35581365
epoch 70, w = -0.191, loss = 0.33250454
epoch 80, w = -0.202, loss = 0.31335399
epoch 90, w = -0.212, loss = 0.29726049
epoch 100, w = -0.221, loss = 0.28348735


In [23]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f"Accuracy = {acc:.3f}")

Accuracy = 0.974
